In [13]:
from data import dataset

##### Split data and batch it in loaders

In [14]:
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

### Model Creation

In [15]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, edge_attr, edge_index):
        x=self.conv1(edge_attr, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.sigmoid(x)
        return x

model = GCN(input_dim=4, hidden_dim=16, output_dim=1)


In [19]:
import torch.optim as optim
import torch.nn.functional as F

optimizer = optim.Adam(model.parameters(), lr=0.01)

criterion = F.binary_cross_entropy_with_logits
num_epochs=5
for epoch in range(num_epochs):
    model.train()
    for data in train_loader:

        optimizer.zero_grad()

        x, edge_index, edge_attr,y = data.x, data.edge_index, data.edge_attr,data.y

        logits = model(edge_attr,edge_index)
        logits = logits.view(-1, 1)
        logits = torch.sigmoid(logits).view(-1)
        loss = criterion(logits, y.view(-1))

        loss.backward()

        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for data in val_loader:
            x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
            y = data.y.float()
            logits = model(edge_attr,edge_index)
            val_loss += criterion(logits.view(-1), y).item()
        val_loss /= len(val_loader)

    print(f"Epoch {epoch}: train_loss = {loss.item()}, val_loss = {val_loss}")


In [18]:
test_loss,correct,total=0,0,0
for batch in test_loader:
 
    x, edge_index, edge_attr, y = batch.x, batch.edge_index, batch.edge_attr, batch.y
    
    with torch.no_grad():
        logits = model(edge_attr,edge_index)
    
    loss = criterion(logits.view(-1), y)
    test_loss += loss.item() * y.size(0)
    
    _, predicted = logits.max(dim=1)
    correct += predicted.eq(y).sum().item()
    
    total += y.size(0)

test_loss /= total
test_acc = 100. * correct / total

print('Test set results: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
    test_loss, correct, total, test_acc))

Test set results: Average loss: 0.6931, Accuracy: 6456474/7847702 (82.27%)
